<a href="https://colab.research.google.com/github/priebet/sentiment/blob/master/sentiment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter-Sentimentanalyse Teil 2
## Erlernen eines NN-Klassifikators mit Kera und TensorFlow

In [0]:
## Mount Google Drive for easy and fast read/write access to data folder 
#from google.colab import drive
#drive.mount('/content/drive')
#datapath = "/content/drive/My Drive/Colab Notebooks/data/"

In [0]:
# For demonstration purposes, pull data from webserver instead
datapath = "http://priebe.onl/data/"

In [2]:
import pandas as pd  

df = pd.read_csv(datapath+"training_cleaned.csv")
df.head()

,text,sentiment
0,awww that bummer you shoulda got david carr of...,0
1,is upset that he can not update his facebook b...,0
2,dived many times for the ball managed to save ...,0
3,my whole body feels itchy and like its on fire,0
4,no it not behaving at all mad why am here beca...,0


In [3]:
from sklearn.model_selection import train_test_split
seed = 2000

x = df['text']
y = df['sentiment']

## Use only 1% sample for demonstration purposes
#_, x, _, y = train_test_split(x, y, test_size=.01, random_state=seed)

x_train, x_validation_test, y_train, y_validation_test = train_test_split(x, y, test_size=.02, random_state=seed)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_test, y_validation_test, test_size=.5, random_state=seed)

print("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_train),
                                                                                         (len(x_train[y_train == 0]) / (len(x_train)*1.))*100,
                                                                                         (len(x_train[y_train == 1]) / (len(x_train)*1.))*100))
print("Validation set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_validation),
                                                                                              (len(x_validation[y_validation == 0]) / (len(x_validation)*1.))*100,
                                                                                              (len(x_validation[y_validation == 1]) / (len(x_validation)*1.))*100))
print("Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_test),
                                                                                        (len(x_test[y_test == 0]) / (len(x_test)*1.))*100,
                                                                                        (len(x_test[y_test == 1]) / (len(x_test)*1.))*100))

Train set has total 1564120 entries with 50.02% negative, 49.98% positive
Validation set has total 15960 entries with 49.45% negative, 50.55% positive
Test set has total 15961 entries with 49.68% negative, 50.32% positive


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(max_features=100000,ngram_range=(1, 3))
tvec.fit(x_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0,
                max_features=100000, min_df=1, ngram_range=(1, 3), norm='l2',
                preprocessor=None, smooth_idf=True, stop_words=None,
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [0]:
x_train_tfidf = tvec.transform(x_train)
x_validation_tfidf = tvec.transform(x_validation)
x_test_tfidf = tvec.transform(x_test)

In [0]:
%tensorflow_version 1.x
from keras.models import Sequential
from keras.layers import Dense

In [0]:
%%time
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=100000))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train_tfidf, y_train, epochs=5, validation_data=(x_validation_tfidf, y_validation), batch_size=1024)

Train on 1564120 samples, validate on 15960 samples
Epoch 1/5
1564120/1564120 [==============================] - 1205s 770us/step - loss: 0.4224 - acc: 0.8122 - val_loss: 0.3897 - val_acc: 0.8254
Epoch 2/5
1564120/1564120 [==============================] - 1124s 718us/step - loss: 0.3687 - acc: 0.8357 - val_loss: 0.3848 - val_acc: 0.8289
Epoch 3/5
1564120/1564120 [==============================] - 1122s 717us/step - loss: 0.3491 - acc: 0.8447 - val_loss: 0.3869 - val_acc: 0.8292
Epoch 4/5
1564120/1564120 [==============================] - 1122s 717us/step - loss: 0.3326 - acc: 0.8527 - val_loss: 0.3932 - val_acc: 0.8283
Epoch 5/5
1564120/1564120 [==============================] - 1122s 717us/step - loss: 0.3163 - acc: 0.8616 - val_loss: 0.4038 - val_acc: 0.8278
CPU times: user 4h 3min 9s, sys: 1min 58s, total: 4h 5min 8s
Wall time: 1h 34min 54s


In [0]:
results = model.evaluate(x_test_tfidf, y_test, batch_size=128)
print('test loss, test acc:', results)

15961/15961 [==============================] - 9s 571us/step
test loss, test acc: [0.41018261473531836, 0.8207505794741383]


In [0]:
from joblib import dump, load

#dump(tvec, datapath+"tvec.pkl")
#model.save(datapath+"model.h5")